# Predicción de series temporales con la clase `SimpleRNN` de Keras
### Dr. Tirthajyoti Sarkar, Fremont, CA 94536 ([LinkedIn](https://www.linkedin.com/in/tirthajyoti-sarkar-2127aa7/), [Github](https://tirthajyoti.github.io))

Para más *notebooks* de estilo tutorial sobre aprendizaje profundo, **[aquí tenéis un repositorio de Github](https://github.com/tirthajyoti/Deep-learning-with-Python)**.

Para más tutoriales sobre aprendizaje automático en general, **[aquí tenéis un repositorio de Github](https://github.com/tirthajyoti/Machine-Learning-with-Python)**.

---
### ¿De qué trata este *notebook*?
En este *notebook*, mostramos la construcción de una simple red neuronal recurrente (Recurrent Neural Network, *RNN*) usando Keras.

Generaremos algunos datos sintéticos de series temporales multiplicando dos señales periódicas/sinusoidales y añadiendo algo de estocasticidad (ruido gaussiano). A continuación, tomaremos una pequeña fracción de los datos y entrenaremos un modelo *RNN* simple con ellos e intentaremos predecir el resto de los datos y ver cómo coinciden las predicciones con la realidad.

In [ ]:
'''
Las RNN son RRNN especializadas en series temporales y en tratamiento de texto.
Llevan un encoding interno de los timestamps utilizados.
LSTM: Long Sort Memory Layer. Son capas capaces de trabajar con secuencias de datos.
Muy útiles para vídeo o speech recognition. Las neuronas tienen una memoria de los
valores anteriores.
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import Callback

In [ ]:
# Cantidad de datos
N = 3004

# Patición train/test.
# 25% de los datos (754) para train y el resto para test
Tp = 754   

np.random.seed(0)
t=np.arange(0,N)
x=(2*np.sin(0.02*t)*np.sin(0.003*t))+0.5*np.random.normal(size=N)
df = pd.DataFrame(x, columns=['Data'])
print(len(df))
df.head()

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(df,c='blue')
plt.grid(True);

### Dividir los valores en *train* y *test*

Así pues, tomamos sólo el 25% de los datos como muestras de entrenamiento y reservamos el resto de los datos para las pruebas. 

Observando el gráfico de la serie temporal, pensamos que **no es fácil que un modelo estándar consiga predicciones correctas**.

In [ ]:
values = df.values
# Train hasta Tp, hasta 754
train, test = values[0:Tp ,:], values[Tp:N,:]

In [ ]:
print("Train data length:", train.shape)
print("Test data length:", test.shape)

In [ ]:
index = df.index.values
plt.figure(figsize=(15,4))
plt.plot(index[0:Tp],train,c='blue')
plt.plot(index[Tp:N],test,c='orange',alpha=0.7)
plt.legend(['Train','Test'])
plt.axvline(Tp, c="r")
plt.grid(True);

### Paso (o *embedding*)
El modelo RNN requiere un valor de $paso$ que contenga número $n$ de elementos como secuencia de entrada.

Supongamos x = {1,2,3,4,5,6,7,8,9,10}

para $step=1$, la entrada $x$ y su predicción $y$ se convierten en:

| $x$ | $y$ |
|---|---|
| 1  | 2  |
| 2  | 3  |
| 3  | 4  |
| ...  | ...  |
| 9  | 10  |

para $step=3$, $x$ e $y$ contienen:

| $x$ | $y$ |
|---|---|
| 1,2,3  | 4  |
| 2,3,4  | 5  |
| 3,4,5  | 6  |
| ...  | ...  |
| 7,8,9  | 10  |

Aquí, elegimos `step=4`. En *RNN* más complejas y, en particular, para el procesamiento de texto, esto también se denomina *embedding size*.

In [ ]:
'''
Básicamente los steps son el número de lags necesarios para realizar las predicciones.
Escogemos 4 lags.
Se los añade al final repetidos porque luego los eliminará al principio, ya que no puede
hacer las predicciones de los lags 1,2,3
'''
train.shape

In [ ]:
df2 = df.copy()
emb_size = 4

'''
Montamos nuevas features con los lags
'''
for i in range(1, emb_size+1):
    df2['lag' + str(i)] = df2['Data'].shift(i)
    
df2.dropna(inplace=True)
df2.reset_index(drop=True, inplace=True)

values = df2.values

'''
Volvemos a montar xtrain, xtest...
'''
trainX,trainY = values[0:Tp-emb_size ,1:],values[0:Tp-emb_size ,0],
testX,testY = values[Tp-emb_size:N-emb_size,1:], values[Tp-emb_size:N-emb_size,0]

print("Train data length:", trainX.shape)
print("Train target length:", trainY.shape)
print("Test data length:", testX.shape)
print("Test target length:", testY.shape)

In [ ]:
df2.head()

In [ ]:
'''
750 son los instantes para entrenar (eq a nº registros)
1 x 4 dimensiones de estos dato, como una imagen. Si fuesen mas variables el 1 seria 2 o 3??
1 es una fila de datos, necesita ese formato
4 los lags para la capa LSTM
'''
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
trainX.shape

In [ ]:
trainX[:3]

Para entrenar el modelo, necesito que los datos tengan la siguiente dimensión:

(750, 1, 4)

- 750: el número total de trozos 
- 1: una fila de datos
- 4: cada trozo tiene cuatro valores

En el caso de una imagen, recordemos con un ejemplo: 

(750, 28, 28)

750 imágenes de resolución 28x28

In [ ]:
print("Training data shape:", trainX.shape,', ',trainY.shape)
print("Test data shape:", testX.shape,', ',testY.shape)

### Modelo de Keras

- 256 neuronas en la capa de la *RNN*.
- 32 neuronas neurons en la capa densamente conectada.
- Una única neurona en la capa de salida. Realice la predicción de un único número.
- Función de activación *ReLu*.
- *Learning Rate*: 0.001

In [ ]:
from tensorflow.keras.layers import Dense, LSTM
'''
embedding es la cantidad de lags utilizada
'''
def build_simple_rnn(num_units=128, embedding=4,num_dense=32,lr=0.001):
    """
    Builds and compiles a simple RNN model
    Arguments:
              num_units: Number of units of a the simple RNN layer
              embedding: Embedding length
              num_dense: Number of neurons in the dense layer followed by the RNN layer
              lr: Learning rate (uses RMSprop optimizer)
    Returns:
              A compiled Keras model.
    """
    model = Sequential()
    # Long short term memory
    # Esto es capa de entrada + capa con 128 neuronas con su función de activacion
    model.add(LSTM(units=num_units, input_shape=(1,embedding), activation="relu"))
    model.add(Dense(num_dense, activation="relu"))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  #optimizer=RMSprop(lr=lr),
                  optimizer='adam',
                  metrics=['mse'])
    
    return model

In [ ]:
model = build_simple_rnn() # Tomando los valores por defecto.
#model.save("my_model.h5")

In [ ]:
'''
Parametros LSTM: 4(nm+n**2+n)
siendo n = nº neuronas y m = embeddings
'''
model.summary()

### Una simple clase callback para mostrar un mensaje cada 50 epochs

In [ ]:
'''
Enseña mensaje si la epoch es multiplo de 50 y no ha acabado de entrenar.
Cada vez que termina una epoch, keras llama a on_epoch_end()
'''
class MyCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1) % 50 == 0 and epoch>0:
            print("Epoch number {} done".format(epoch+1))

### Ajuste del modelo
Con `batch_size` = 16 lo que haríamos es que cogemos los datos de esta forma:

- (16, 1, 4)

Cogemos 16 trozos de 1 fila con 4 datos

In [ ]:
batch_size=16
num_epochs = 1000

In [ ]:
model.fit(trainX,trainY, 
          epochs=num_epochs, 
          batch_size=batch_size, 
          callbacks=[MyCallback()],verbose=0)

### Visualización de la función de perdida

In [ ]:
'''
Va bajando mucho el RMSE con las epochs, incluso podriamos probar mas epochs
'''
plt.figure(figsize=(7,5))
plt.title("RMSE loss over epochs",fontsize=16)
plt.plot(np.sqrt(model.history.history['loss']),c='k',lw=2)
plt.grid(True)
plt.xlabel("Epochs",fontsize=14)
plt.ylabel("Root-mean-squared error",fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14);

### Predicciones
Observa que el modelo se ha ajustado solo con `trainX` y `trainY`. 

In [ ]:
trainX

In [ ]:
plt.figure(figsize=(5,4))
plt.title("This is what the model saw",fontsize=18)
# Saca todos los valores de train, la primera columna
plt.plot(trainX[:,0],c='blue')
plt.grid(True)
plt.show()

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
predicted = np.concatenate((trainPredict,testPredict),axis=0)

In [ ]:
plt.figure(figsize=(10,4))
plt.title("This is what the model predicted",fontsize=18)
plt.plot(testPredict,c='orange')
plt.grid(True)
plt.show()

### Comparación con el conjunto *test*

In [ ]:
index = df2.index.values
plt.figure(figsize=(15,4))
plt.title("Ground truth and prediction together",fontsize=18)
plt.plot(index,df2,c='blue')
plt.plot(index,predicted,c='orange',alpha=0.75)
plt.legend(['True data','Predicted'],fontsize=15)
plt.axvline(df.index[Tp], c="r")
plt.grid(True)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

### ¿Cómo se distribuyen los errores?
Los errores, o residuos, como se denominan en un problema de regresión, pueden representarse gráficamente para ver si siguen alguna distribución específica. En el proceso de generación, inyectamos ruido gaussiano, por lo que esperamos que el error siga el mismo patrón, *si el modelo ha sido capaz de ajustarse a los datos reales correctamente*.

In [ ]:
'''
Si encontramos patrones raros en los residuos es porque el modelo no se ha ajustado
bien a los datos. Habria que probar otras configuraciones/modelos.
'''
error = predicted[Tp:N]-df2[Tp:N]
# Ravel elimina una dimension, lo aplana todo. Como flatten
error = np.array(error).ravel()

In [ ]:
plt.figure(figsize=(7,5))
plt.hist(error,bins=25,edgecolor='k',color='orange')
plt.show()

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(error,c='blue',alpha=0.75)
plt.hlines(y=0,xmin=-50,xmax=2400,color='k',lw=3)
plt.xlim(-50,2350)
plt.grid(True);

## Mejorar el modelo

Tenga en cuenta que, para que estos experimentos sean razonablemente rápidos, fijaremos el tamaño del modelo para que sea más pequeño que el modelo anterior. Utilizaremos una capa *RNN* de 32 neuronas seguida de una capa densamente conectada de 8 neuronas.

### Variando el tamaño `embedding`/`step`

In [ ]:
def predictions(model,trainX,testX):
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    predicted = np.concatenate((trainPredict,testPredict),axis=0)
    
    return predicted

In [ ]:
def plot_compare(predicted, df2):
    index = df2.index.values
    plt.figure(figsize=(15,4))
    plt.title("Ground truth and prediction together",fontsize=18)
    plt.plot(index,df2,c='blue')
    plt.plot(index,predicted,c='orange',alpha=0.75)
    plt.legend(['True data','Predicted'],fontsize=15)
    plt.axvline(df2.index[Tp], c="r")
    plt.grid(True)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.show()

In [ ]:
def prepare_data(step=4):
    df2 = df.copy()
    emb_size = step
    for i in range(1, emb_size+1):
        df2['lag' + str(i)] = df2['Data'].shift(i)

    df2.dropna(inplace=True)
    df2.reset_index(drop=True, inplace=True)

    values = df2.values

    trainX,trainY = values[0:Tp-emb_size ,1:],values[0:Tp-emb_size ,0],
    testX,testY = values[Tp-emb_size:N-emb_size,1:], values[Tp-emb_size:N-emb_size,0]
    
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    
    return trainX,testX,trainY,testY,df2

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
def errors(testX, testY):
    y_true = testY
    y_pred = model.predict(testX)
    return mean_absolute_error(y_true=y_true, y_pred=y_pred)

In [ ]:
'''
Parece que cuanto mayor es la ventana, peor le está viniendo al modelo
Mas ruido le mete. Tb es cierto, para 100 epochs
'''
for s in [2,4,6,8,10,12]:
    trainX,testX,trainY,testY,df2 = prepare_data(s)
    model = build_simple_rnn(num_units=32,num_dense=8,embedding=s)
    batch_size=16
    num_epochs = 100
    model.fit(trainX,trainY, 
          epochs=num_epochs, 
          batch_size=batch_size,
          verbose=0)
    preds = predictions(model,trainX,testX)
    print("Embedding size: {}".format(s))
    print("MAE:", errors(testX, testY))
    print("-"*100)
    plot_compare(preds, df2)
    print()

### Número de `epochs`

In [ ]:
'''
Probemos ahora con una ventana grande (8), y unas cuantas epochs mas
'''
for e in [100,200,300,400,500]:
    trainX, testX, trainY, testY, df2 = prepare_data(6)
    model = build_simple_rnn(num_units=32,num_dense=8,embedding=6)
    batch_size=16
    num_epochs = e
    model.fit(trainX,trainY, 
          epochs=e, 
          batch_size=batch_size,
          verbose=0)
    preds = predictions(model,trainX,testX)
    print("Ran for {} epochs".format(e))
    print("MAE:", errors(testX, testY))
    print("-"*100)
    plot_compare(preds, df2)
    print()

### *Batch size* (tamaño del lote)

In [ ]:
for b in [4,8,16,32,64,128]:
    trainX,testX,trainY,testY, df2 = prepare_data(6)
    model = build_simple_rnn(num_units=32,num_dense=8,embedding=6)
    batch_size=b
    num_epochs = 100
    model.fit(trainX,trainY, 
          epochs=num_epochs, 
          batch_size=b,
          verbose=0)
    preds = predictions(model,trainX,testX)
    print("Ran with batch size: {}".format(b))
    print("MAE:", errors(testX, testY))
    print("-"*100)
    plot_compare(preds, df2)
    print()

### Resumen

Claramente, se observaron las siguientes tendencias,

- Un tamaño de *embedding* demasiado pequeño no es útil, pero un *embedding* muy larga tampoco es eficaz. Un *embedding* de 8 parece buena para estos datos.
- Un mayor número de *epochs* no siempre es mejor. Probablemente estamos sufriendo un exceso de ajuste.
- Un *batch size* de 32 o 64 parece óptimo.

En última instancia, es necesario un ajuste exhaustivo de los hiperparámetros para obtener el mejor rendimiento global.

------------------------------------------------------
## Caso de uso
* NETFLIX --> nflx
* TESLA --> tsla
* GOOGLE --> goog  
* META --> meta 
* APPLE --> aapl 
* AMAZON --> amzn 
* IBEX 35 --> ^ibex 
* PETROLEO --> bz=f 
* ORO --> gc=f
* PLATA --> si=f
* EURO/USD --> eurusd=x
* REPSOL --> rep.mc
* IBERDROLA --> ibe.mc
* ETHEREUM --> eth-usd 


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import Callback

In [ ]:
nflx = yf.download("nflx", start='2020-01-01', end='2023-06-30')
df_nflx = pd.DataFrame(nflx)

display(df_nflx.shape, df_nflx.head())

In [ ]:
values = df_nflx.Close
train, test = values[:865], values[865:]

In [ ]:
print("Train data length:", train.shape)
print("Test data length:", test.shape)

index = df_nflx.index.values
plt.figure(figsize=(15,4))
plt.plot(index[0:865],train,c='blue')
plt.plot(index[865:],test,c='orange',alpha=0.7)
plt.legend(['Train','Test'])
#plt.axvline(865, c="r")
plt.grid(True);

In [ ]:
model = build_simple_rnn()
model.summary()